In [1]:

from keras.models import Sequential  # Modelo sequencial
from keras.layers import Dense, Flatten  # 3ª e 4ª etapa da CNN
from keras.layers import Conv2D, MaxPooling2D  # 1ª e 2ª etapa da CNN
from keras.layers import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dropout
from keras.preprocessing import image
from keras import utils
from sklearn.model_selection import StratifiedKFold
import numpy as np
import pandas as pd
import cv2 
import os 
import glob


In [ ]:
'''
    Este trecho do código tem a função de para mudar a semente geradora dos
    numeros aleatórios. Será utilizado para obter os mesmos resultados ao
    fazer manualmente a divisão da base de dados durante a operação
    de validação cruzada
'''
seed = 5
np.random.seed(seed)

In [2]:
''' 
    Carregando a pasta de imagens (Dataset dos ovos)
    * A função glob encontra todos os nomes de caminhos que correspondem a um padrão 
      especificado de acordo com as regras usadas pelo shell Unix

    * Para cada um dos arquivos encontrados é feita a leitura
      do arquivo .jpg (imagem), as imagens lidas são armazenadas
      na lista 'data'
'''

img_dir = "C:/Users/marci/Desktop/Projeto mestrado/CNN Egg application/Egg Dataset"
data_path = os.path.join(img_dir,'*g') 

folder = glob.glob(data_path) 
data = [] 
for files in folder: 
    img = cv2.imread(files) 
    data.append(img) 

In [3]:
'''
    Carregar a base de dados
    Obs: para a validação cruzada, apenas os valores de treinamento serão utilizadas,
    ou seja -> (xTrain, yTrain)
'''

xTrain = np.array(data)
yTrain = pd.read_csv("RealData.csv")

print('Shape dos atributos previsores de treinamento', xTrain.shape)
print('Shape das classes de treinamento', yTrain.shape)


Shape dos atributos previsores de treinamento (5100, 512, 512, 3)
Shape das classes de treinamento (5100, 1)


In [32]:
'''
    A variável kfold busca controlar a validação cruzada, a variável recebe
    a função StratifiedKFold, com os seguintes parâmetros:

    * n_splits -> Quantidade de 'dobras' ou seções em que a base de dados será dividida
    * shuffle -> Define que os dados serão analizados aleatóriamente
    * random_state -> Quando 'shuffle' for setado como true, o random_state afeta a ordem dos indices
        controlando a aleatoriedade de cada dobra para cada classe
'''

kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed)

'''
    O resultado de cada execução é inserido na lista da variável results
'''
results = []

In [35]:
'''
    O loop tem o objetivo de percorrer os dados selecionados para treinamento e teste em cada iteração,
    utilizando a variável kfold (criada préviamente para controlar a validação cruzada).

    'trainIndex' e 'testIndex' são definidos pelos ÍNDICES da base de dados que serão utilizados
    para realizar treinamento da rede neural em cada iteração. 
    Seus valores serão definidos pela divisão de kfold e serão sempre distintos, uma vez que um grupo
    será utilizado para treinamento e outro para teste.
'''

for trainIndex, testIndex in kfold.split(xTrain, yTrain):
   
    '''
        Definição de uma CNN básica
    '''   
    classificador = Sequential()
    classificador.add(Conv2D(32, (3,3), activation = 'relu'))
    classificador.add(MaxPooling2D(pool_size = (2,2)))
    classificador.add(Flatten())

    classificador.add(Dense(units = 128, activation = 'relu'))
    classificador.add(Dense(units = 10, activation = 'softmax'))
    classificador.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

    '''
        O treinamento da rede neural se dá por meio dos dados definidos pelo index definidos em kfold
    '''
    classificador.fit(xTrain[trainIndex], yTrain[trainIndex], 
                      batch_size = 128, epochs = 5)
    
    '''
        O método evaluate retorna a acurácia do treinamento em cada iteração, tais resultados são armazenados
        na lista 'resultados'
    '''
    precision = classificador.evaluate(xTrain[testIndex], yTrain[testIndex])

    results.append(precision[1])

KeyError: "None of [Index([   0,    2,    3,    4,    5,    6,    7,    8,   10,   11,\n       ...\n       5088, 5089, 5090, 5091, 5092, 5093, 5094, 5095, 5096, 5097],\n      dtype='int32', length=4080)] are in the [columns]"